In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from matplotlib import style
from scipy import arange
style.use('fivethirtyeight')
%matplotlib inline

In [ ]:
df = pd.read_excel('dbscan.xlsx',sheet_name='Sheet1')

In [ ]:
df.head()

In [ ]:
# set index to account number and month
df = df.set_index(['',''])

In [ ]:
# check for blank entries
df = df.replace('NONE',np.nan)
df = df.replace('-', np.nan)
df = df.dropna()

In [ ]:
len(df)

In [ ]:
df

In [ ]:
table = dict.fromkeys([0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,0.50,
                       1.00,1.5,2.00,2.5,3.00,3.5,4.00,4.5,5.00])

In [ ]:
for i in arange(0.05, 0.55, 0.05):
    
    dbscan = DBSCAN(eps=i, min_samples=500, n_jobs=-1)
    scaler = StandardScaler()
    scaled_data1 = scaler.fit_transform(df)
    dbscan.fit(scaled_data1.data)

    labels = dbscan.labels_

    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_0 = list(labels).count(0)
    n_1 = list(labels).count(1)
    n_noise_ = list(labels).count(-1)
    
    insert = [n_clusters_ ,n_noise_ , n_0 , n_1]
    table[i] = insert
    
    print('Using epsilon of: %f' % i)
    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of 0: %d' % n_0)
    print('Estimated number of 1: %d' % n_1)
    print('Estimated number of noise points: %d' % n_noise_)


In [ ]:

i = 1.5
dbscan = DBSCAN(eps=i, min_samples=100, n_jobs=-1)
scaler = StandardScaler()
scaled_data1 = scaler.fit_transform(df)
dbscan.fit(scaled_data1.data)
labels = dbscan.labels_

n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_0 = list(labels).count(0)
n_1 = list(labels).count(1)
n_noise_ = list(labels).count(-1)
    
insert = [n_clusters_ ,n_noise_ , n_0 , n_1]
table[i] = insert

| eps | n_clusters | n_noise | n_0 | n_1 |
| --- | --- | --- | --- | --- |
| 0.05 | 2 | 129429|64206 |7197 |
| 0.10 | 2 | 98021 |95537 |7274 |
| 0.15 | 3 | 94270 |98083 |7345 |
| 0.20 | 3 | 81577| 99976 |7454 |
| 0.25 | 2 | 68635| 124638 |7559|
| 0.30 | 2 | 59548| 133638| 7646|
| 0.35 | 4 | 48079| 140451| 7742|
| 0.40 | 2 | 35608| 157331| 7893|
| 0.45 | 2 | 29330| 163447| 8055|
| 0.50 | 2 | 25247| 167496| 8089|
| 1.0  | | | |  |
| 1.5  | 2 | 1703| 198953| 176|

In [ ]:
table[0.15]=[3, 94270, 98083, 7345]
table[0.35]=[4, 48079, 140451, 7742]

In [ ]:
table

In [ ]:
df['clusters'] = labels

In [ ]:
test = df[df['clusters'] == -1]

In [ ]:
test = test.drop(columns='index')

In [ ]:
test

In [ ]:
test = test.reset_index()

In [ ]:
# check if account numbe is unique
len(test[''].unique())

In [ ]:
train = df[df['clusters'] != -1]

In [ ]:
train = train.drop(columns='index')

In [ ]:
train

In [ ]:
writer = pd.ExcelWriter('train_index.xlsx')
train.to_excel(writer,'Sheet1',index=True)
writer.save()

In [ ]:
writer = pd.ExcelWriter('test_index.xlsx')
test.to_excel(writer,'Sheet1',index=True)
writer.save()

In [ ]:

from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=5, n_jobs=-1)
neigh.fit(train)

In [ ]:
knn_dist, knn_ind = neigh.kneighbors(test, return_distance=True) 

In [ ]:
len(knn_dist)

In [ ]:
np.set_printoptions(threshold=1000)

In [ ]:
count = 0
avg = [] 
for val in knn_dist:
    #avg = np.average(val)
    #print(avg)
    avg.append(np.average(val))
    count += 1

In [ ]:
len(avg)

In [ ]:
test['AVG_distance'] = avg

In [ ]:
test[["AVG_distance"]].sort_values(["AVG_distance"], ascending=False)

In [ ]:
knn_ind

In [ ]:
points = pd.DataFrame(knn_ind)

In [ ]:
points

In [ ]:
neighbour1 = points.iloc[:,0]
neighbour1 = pd.DataFrame(neighbour1)

neighbour2 = points.iloc[:,1]
neighbour2 = pd.DataFrame(neighbour2)

neighbour3 = points.iloc[:,2]
neighbour3 = pd.DataFrame(neighbour3)

neighbour4 = points.iloc[:,3]
neighbour4 = pd.DataFrame(neighbour4)

neighbour5 = points.iloc[:,4]
neighbour5 = pd.DataFrame(neighbour5)

In [ ]:
neighbour1

In [ ]:
neighbour2

In [ ]:
neighbour3

In [ ]:
neighbour4

In [ ]:
neighbour5

In [ ]:
neighbour_feat = train.reset_index()

In [ ]:
neighbour_feat

## Join neighbour row number df with features on the same row number 

In [ ]:
neighbour1 = neighbour1.rename(columns={0: 'row number'})
neighbour1 = neighbour1.set_index('row number')

neighbour2 = neighbour2.rename(columns={1: 'row number'})
neighbour2 = neighbour2.set_index('row number')

neighbour3 = neighbour3.rename(columns={2: 'row number'})
neighbour3 = neighbour3.set_index('row number')

neighbour4 = neighbour4.rename(columns={3: 'row number'})
neighbour4 = neighbour4.set_index('row number')

neighbour5 = neighbour5.rename(columns={4: 'row number'})
neighbour5 = neighbour5.set_index('row number')

In [ ]:
neighbour1 = pd.concat([neighbour1, neighbour_feat],axis=1, join='inner')
neighbour2 = pd.concat([neighbour2, neighbour_feat],axis=1, join='inner')
neighbour3 = pd.concat([neighbour3, neighbour_feat],axis=1, join='inner')
neighbour4 = pd.concat([neighbour4, neighbour_feat],axis=1, join='inner')
neighbour5 = pd.concat([neighbour5, neighbour_feat],axis=1, join='inner')

In [ ]:
neighbour1

In [ ]:
neighbour2

In [ ]:
neighbour3

In [ ]:
neighbour4

In [ ]:
neighbour5

## Join all 5 neighbour frames

In [ ]:
# join using same account number and month
neighbour1 = neighbour1.reset_index()
neighbour1 = neighbour1.rename(columns={'index': 'Neighbour_1', 
                                        '': '_1',
                                        '': '_1',
                                        'TOTALSPENDNORM': 'TOTALSPENDNORM_1',
                                        'AVGSPENDNORM': 'AVGSPENDNORM_1',
                                        'TRANSNUMNORM': 'TRANSNUMNORM_1',
                                        'MAXSPENDNORM': 'MAXSPENDNORM_1',
                                        'MAXEARNNORM': 'MAXEARNNORM_1',
                                        'AVGEARNNORM': 'AVGEARNNORM_1',
                                        'MAXBURNNORM': 'MAXBURNNORM_1',
                                        'AVGBURNNORM': 'AVGBURNNORM_1',
                                        'clusters': 'clusters_1'
                                       })

In [ ]:
neighbour1

In [ ]:
# join using same account number and month
neighbour2 = neighbour2.reset_index()
neighbour2 = neighbour2.rename(columns={'index': 'Neighbour_2', 
                                        '': '_2',
                                        '': '_2',
                                        'TOTALSPENDNORM': 'TOTALSPENDNORM_2',
                                        'AVGSPENDNORM': 'AVGSPENDNORM_2',
                                        'TRANSNUMNORM': 'TRANSNUMNORM_2',
                                        'MAXSPENDNORM': 'MAXSPENDNORM_2',
                                        'MAXEARNNORM': 'MAXEARNNORM_2',
                                        'AVGEARNNORM': 'AVGEARNNORM_2',
                                        'MAXBURNNORM': 'MAXBURNNORM_2',
                                        'AVGBURNNORM': 'AVGBURNNORM_2',
                                        'clusters': 'clusters_2'
                                       })

In [ ]:
neighbour2

In [ ]:
# join using same account number and month
neighbour3 = neighbour3.reset_index()
neighbour3 = neighbour3.rename(columns={'index': 'Neighbour_3', 
                                        '': '_3',
                                        '': '_3',
                                        'TOTALSPENDNORM': 'TOTALSPENDNORM_3',
                                        'AVGSPENDNORM': 'AVGSPENDNORM_3',
                                        'TRANSNUMNORM': 'TRANSNUMNORM_3',
                                        'MAXSPENDNORM': 'MAXSPENDNORM_3',
                                        'MAXEARNNORM': 'MAXEARNNORM_3',
                                        'AVGEARNNORM': 'AVGEARNNORM_3',
                                        'MAXBURNNORM': 'MAXBURNNORM_3',
                                        'AVGBURNNORM': 'AVGBURNNORM_3',
                                        'clusters': 'clusters_3'
                                       })

In [ ]:
neighbour3

In [ ]:
# join using same account number and month
neighbour4 = neighbour4.reset_index()
neighbour4 = neighbour4.rename(columns={'index': 'Neighbour_4', 
                                        '': '_4',
                                        '': '_4',
                                        'TOTALSPENDNORM': 'TOTALSPENDNORM_4',
                                        'AVGSPENDNORM': 'AVGSPENDNORM_4',
                                        'TRANSNUMNORM': 'TRANSNUMNORM_4',
                                        'MAXSPENDNORM': 'MAXSPENDNORM_4',
                                        'MAXEARNNORM': 'MAXEARNNORM_4',
                                        'AVGEARNNORM': 'AVGEARNNORM_4',
                                        'MAXBURNNORM': 'MAXBURNNORM_4',
                                        'AVGBURNNORM': 'AVGBURNNORM_4',
                                        'clusters': 'clusters_4'
                                       })

In [ ]:
neighbour4

In [ ]:
# join using same account number and month
neighbour5 = neighbour5.reset_index()
neighbour5 = neighbour5.rename(columns={'index': 'Neighbour_5', 
                                        '': '_5',
                                        '': '_5',
                                        'TOTALSPENDNORM': 'TOTALSPENDNORM_5',
                                        'AVGSPENDNORM': 'AVGSPENDNORM_5',
                                        'TRANSNUMNORM': 'TRANSNUMNORM_5',
                                        'MAXSPENDNORM': 'MAXSPENDNORM_5',
                                        'MAXEARNNORM': 'MAXEARNNORM_5',
                                        'AVGEARNNORM': 'AVGEARNNORM_5',
                                        'MAXBURNNORM': 'MAXBURNNORM_5',
                                        'AVGBURNNORM': 'AVGBURNNORM_5',
                                        'clusters': 'clusters_5'
                                       })

In [ ]:
neighbour5

In [ ]:
neighbours = pd.concat([neighbour1,neighbour2,neighbour3,neighbour4,neighbour5], axis=1, sort=False)

In [ ]:
neighbours

In [ ]:
for i in range(1,6):
    i = str(i)
    neighbours = neighbours.drop(['Neighbour_'+i, 'ACCOUNTNUM_'+i, 'MONTH_'+i, 'clusters_'+i], axis=1)
display(neighbours)

## Join neighbours features with test frame

In [ ]:
result = pd.concat([test.reset_index(), neighbours], axis=1)

In [ ]:
test

In [ ]:
result

## find average from difference of test's feature - neighbours feature

In [ ]:
result.pop('clusters')

In [ ]:
result.columns

In [ ]:
# set index to account number and month
to_process = result.set_index(['', ''])

In [ ]:
for c in to_process.columns[:8]:
    delta_f = (to_process[c]-to_process[c+'_1'])+(to_process[c]-to_process[c+'_2'])+(to_process[c]-to_process[c+'_3'])+(to_process[c]-to_process[c+'_4'])+(to_process[c]-to_process[c+'_5'])
    avg_delta_f = delta_f / 5
    to_process[c+'_avgdiff'] = avg_delta_f

In [ ]:
to_process

In [ ]:
rank = to_process.iloc[:, [8,49,50,51,52,53,54,55,56]]

## Rank features

In [ ]:
rank

In [ ]:
rank.sort_values('AVG_distance',ascending=False).filter(like='avgdiff').rank(axis=1, method='dense')

In [ ]:
list(test.sort_values(["AVG_distance"], ascending=False).reset_index()['ACCOUNTNUM']).index('9200145768')

In [ ]:
dbscan_top30 = rank.sort_values('AVG_distance',ascending=False).filter(like='avgdiff').rank(axis=1, method='dense').reset_index()

In [ ]:
dbscan_top30 = dbscan_top10[0:30].reset_index()

In [ ]:
dbscan_top30

In [ ]:
writer = pd.ExcelWriter('dbscanTop30.xlsx')
dbscan_top30.to_excel(writer,'Sheet1',index=True)
writer.save()

In [ ]:
dbscan_result = rank.sort_values('AVG_distance',ascending=False).filter(like='avgdiff').rank(axis=1, method='dense').reset_index()

In [ ]:
dbscan_result = dbscan_result.reset_index()

In [ ]:
dbscan_result

In [ ]:
dbscan_result.head()

In [ ]:
writer = pd.ExcelWriter('dbscan_result.xlsx')
dbscan_result.to_excel(writer,'Sheet1',index=False)
writer.save()

In [ ]:
dbscan_result